In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [11]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.135522                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.266147
0.000750      2.0  0.011075                  2.0  0.084551
0.001052      3.0  0.008958                  3.0  0.045136
0.001476      4.0  0.003867                  4.0  0.008405
0.002070      5.0 -0.000992                  5.0 -0.018635
0.002904      6.0 -0.005214                  6.0 -0.033799
0.004074      7.0 -0.009058                  7.0 -0.040326
0.005714      8.0 -0.012832                  8.0 -0.041666
0.008015      9.0 -0.016731                  9.0 -0.042190
0.011243     10.0 -0.020919                 10.0 -0.042417
0.015771     11.0 -0.025537                 11.0 -0.043557
0.022122     12.0 -0.030826                 12.0 -0.046094
0.031031     13.0 -0.037804                 13.0 -0.051109
0.043528     14.0 -0.050266                 14.0 -0.062983
0.061057     15.0 -0.058666                 15.0 -0.068743
0.085645     16.0 -0.056174                 16.0 -0.058880
0.120136     17.0 -0.049593                 17.0 -0.041140
0.168516     18.0 -0.040878                 18.0 -0.021598
0.236378     19.0 -0.025044                 19.0 -0.002212
0.331549     20.0  0.003612                 20.0  0.016888
0.465100     21.0  0.061662                 21.0  0.052462
0.652400     22.0  0.178621                 22.0  0.151081
0.915100     23.0  0.208920                 23.0  0.174588
1.283650     24.0  0.148117                 24.0  0.137264
1.800600     25.0  0.102421                 25.0  0.116980
2.525700     26.0  0.060482                 26.0  0.085680
3.542800     27.0  0.035792                 27.0  0.056594
4.969550     28.0  0.021337                 28.0  0.028832
6.970850     29.0  0.011901                 29.0  0.009273
9.778100     30.0  0.005698                 30.0  0.003391
13.715850    31.0  0.001829                 31.0  0.004507
19.239350    32.0 -0.000290                 32.0  0.003433
26.987250    33.0 -0.001131                 33.0  0.000832
37.855300    34.0 -0.001156                 34.0 -0.000867
53.100050    35.0 -0.001794                 35.0 -0.001980
73.887500    36.0 -0.002404                 36.0 -0.002460
97.662500    37.0 -0.002777                 37.0 -0.002793
121.437500   38.0 -0.003084                 38.0 -0.003088
145.212500   39.0 -0.003142                 39.0 -0.003105
168.987500   40.0 -0.003400                 40.0 -0.003489
192.762500   41.0 -0.003643                 41.0 -0.003842
216.537500   42.0 -0.003854                 42.0 -0.004173
240.312500   43.0 -0.004027                 43.0 -0.004423
264.087500   44.0 -0.004161                 44.0 -0.004633
287.862500   45.0 -0.004257                 45.0 -0.004770
311.637500   46.0 -0.004313                 46.0 -0.004813
335.412500   47.0 -0.004332                 47.0 -0.004813
359.187500   48.0 -0.004311                 48.0 -0.004729
382.962500   49.0 -0.004252                 49.0 -0.004585
406.737500   50.0 -0.004153                 50.0 -0.004369
430.512500   51.0 -0.004013                 51.0 -0.004091
454.287500   52.0 -0.003833                 52.0 -0.003752
478.062500   53.0 -0.003612                 53.0 -0.003392
501.837500   54.0 -0.003347                 54.0 -0.002902
525.612500   55.0 -0.003035                 55.0 -0.002441
549.387500   56.0 -0.002672                 56.0 -0.001864
573.162500   57.0 -0.002251                 57.0 -0.001238
596.937500   58.0 -0.001750                 58.0 -0.000538
620.712500   59.0 -0.001134                 59.0  0.000273
644.487500   60.0 -0.000318                 60.0  0.001260
668.262500   61.0  0.000778                 61.0  0.002458
692.037500   62.0  0.002005                 62.0  0.003680
715.812500   63.0  0.002763                 63.0  0.004256
739.587500   64.0  0.002633                 64.0  0.003931
76

# Fluxes by Layer

CRD                        CLIRAD                    
                        flug      fldg     fnetg      flug      fldg     fnetg
pressure     level                                                            
0.000000e+00 1     -6.709430  0.000000 -6.709430       NaN       NaN       NaN
1.000000e-08 1           NaN       NaN       NaN -6.609525  0.000002 -6.609523
6.244000e-04 2     -6.709434  0.000014 -6.709420 -6.609533  0.000029 -6.609504
8.759000e-04 3     -6.709436  0.000017 -6.709420 -6.609536  0.000035 -6.609501
1.228600e-03 4     -6.709439  0.000020 -6.709419 -6.609540  0.000041 -6.609499
1.723400e-03 5     -6.709443  0.000024 -6.709419 -6.609546  0.000047 -6.609499
2.417400e-03 6     -6.709448  0.000029 -6.709419 -6.609553  0.000053 -6.609500
3.390900e-03 7     -6.709454  0.000034 -6.709420 -6.609563  0.000059 -6.609504
4.756500e-03 8     -6.709462  0.000041 -6.709421 -6.609576  0.000065 -6.609511
6.672000e-03 9     -6.709474  0.000050 -6.709424 -6.609594  0.000074 -6.609520
9.358900e-03 10    -6.709490  0.000061 -6.709429 -6.609619  0.000085 -6.609534
1.312780e-02 11    -6.709513  0.000074 -6.709439 -6.609653  0.000100 -6.609552
1.841450e-02 12    -6.709544  0.000090 -6.709455 -6.609701  0.000121 -6.609580
2.583020e-02 13    -6.709590  0.000108 -6.709482 -6.609769  0.000149 -6.609620
3.623230e-02 14    -6.709658  0.000130 -6.709528 -6.609871  0.000188 -6.609683
5.082340e-02 15    -6.709772  0.000157 -6.709615 -6.610033  0.000240 -6.609792
7.129060e-02 16    -6.709966  0.000209 -6.709757 -6.610291  0.000333 -6.609959
1.000000e-01 17    -6.710276  0.000328 -6.709948 -6.610680  0.000521 -6.610159
1.402710e-01 18    -6.710752  0.000567 -6.710185 -6.611242  0.000887 -6.610355
1.967600e-01 19    -6.711466  0.001008 -6.710459 -6.612045  0.001545 -6.610500
2.759970e-01 20    -6.712509  0.001815 -6.710694 -6.613172  0.002651 -6.610521
3.871000e-01 21    -6.713938  0.003292 -6.710646 -6.614688  0.004390 -6.610298
5.431000e-01 22    -6.715495  0.005988 -6.709507 -6.616382  0.007053 -6.609329
7.617000e-01 23    -6.715462  0.010580 -6.704881 -6.616386  0.010970 -6.605416
1.068500e+00 24    -6.712944  0.015655 -6.697289 -6.613892  0.014821 -6.599070
1.498800e+00 25    -6.709108  0.019369 -6.689739 -6.610157  0.018084 -6.592073
2.102400e+00 26    -6.704374  0.021959 -6.682416 -6.605531  0.021823 -6.583708
2.949000e+00 27    -6.699595  0.023245 -6.676350 -6.600707  0.025593 -6.575115
4.136600e+00 28    -6.695038  0.023723 -6.671315 -6.595840  0.028687 -6.567152
5.802500e+00 29    -6.690734  0.023630 -6.667104 -6.590974  0.029512 -6.561462
8.139200e+00 30    -6.686743  0.022933 -6.663810 -6.586135  0.027240 -6.558895
1.141700e+01 31    -6.683157  0.021559 -6.661598 -6.581487  0.023909 -6.557578
1.601470e+01 32    -6.680098  0.019496 -6.660602 -6.577263  0.022139 -6.555123
2.246400e+01 33    -6.677717  0.016893 -6.660824 -6.573667  0.021167 -6.552501
3.151050e+01 34    -6.676143  0.014108 -6.662035 -6.570724  0.019115 -6.551609
4.420010e+01 35    -6.675254  0.011482 -6.663773 -6.568099  0.015186 -6.552913
6.200000e+01 36    -6.676157  0.008602 -6.667555 -6.567128  0.010040 -6.557087
8.577500e+01 37    -6.680320  0.005994 -6.674326 -6.570152  0.006136 -6.564016
1.095500e+02 38    -6.686934  0.004788 -6.682146 -6.576671  0.004787 -6.571883
1.333250e+02 39    -6.696104  0.005272 -6.690832 -6.586511  0.005930 -6.580581
1.571000e+02 40    -6.707379  0.007698 -6.699680 -6.598691  0.009366 -6.589326
1.808750e+02 41    -6.720850  0.011593 -6.709256 -6.613244  0.014090 -6.599154
2.046500e+02 42    -6.736595  0.017079 -6.719516 -6.630231  0.020255 -6.609977
2.284250e+02 43    -6.754673  0.024302 -6.730371 -6.649776  0.028045 -6.621731
2.522000e+02 44    -6.775111  0.033399 -6.741712 -6.671847  0.037657 -6.634190
2.759750e+02 45    -6.797929  0.044498 -6.753431 -6.696490  0.049252 -6.647238
2.997500e+02 46    -6.823134  0.057715 -6.765419 -6.723718  0.063046 -6.660673
3.235250e+02 47    -6.850720  0.073153 -6.777567 -6.753444  0.079214 -6.674

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')